# Texi Guru Kaggle Challange
* Step 1: Data importing

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns


from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

##### Reading test data and train data

In [2]:
try: # For kaggle
  train_pd = pd.read_csv('/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/train.csv')

  test_pd = pd.read_csv('/kaggle/input/taxi-fare-guru-total-amount-prediction-challenge/test.csv')
  print('running on kaggle')
except: #for local machine 
  print('running on local machine')
  train_pd = pd.read_csv('train.csv')
  test_pd = pd.read_csv('test.csv')
  pass
copy_train_pd = train_pd.copy()
copy_test_pd = test_pd.copy()
train_pd.info()


running on local machine
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175000 entries, 0 to 174999
Data columns (total 17 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   VendorID               175000 non-null  int64  
 1   tpep_pickup_datetime   175000 non-null  object 
 2   tpep_dropoff_datetime  175000 non-null  object 
 3   passenger_count        168923 non-null  float64
 4   trip_distance          175000 non-null  float64
 5   RatecodeID             168923 non-null  float64
 6   store_and_fwd_flag     168923 non-null  object 
 7   PULocationID           175000 non-null  int64  
 8   DOLocationID           175000 non-null  int64  
 9   payment_type           175000 non-null  object 
 10  extra                  175000 non-null  float64
 11  tip_amount             175000 non-null  float64
 12  tolls_amount           175000 non-null  float64
 13  improvement_surcharge  175000 non-null  float64
 14  total_amoun

##### Computing the null values and missing values from the tables

In [3]:
train_pd.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,extra,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-06-28 17:20:21,2023-06-28 16:34:45,1.0,2.14,1.0,N,120,9,Credit Card,2.5,7.165589,0.0,1.0,20.64,2.5,0.0
1,0,2023-06-29 23:05:01,2023-06-29 22:01:35,1.0,2.70,1.0,N,15,215,Credit Card,3.5,6.067401,0.0,1.0,25.55,2.5,0.0
2,1,2023-06-30 10:19:31,2023-06-30 11:13:10,1.0,1.15,1.0,N,167,223,Credit Card,0.0,4.111547,0.0,1.0,17.64,2.5,0.0
3,0,2023-06-29 13:23:09,2023-06-29 14:20:01,1.0,0.40,1.0,N,128,239,Credit Card,2.5,6.411079,0.0,1.0,12.80,2.5,0.0
4,1,2023-06-29 22:03:32,2023-06-29 22:22:22,3.0,1.10,1.0,N,203,52,Credit Card,1.0,4.769377,0.0,1.0,18.00,2.5,0.0


In [4]:
train_pd.isna().sum()

VendorID                    0
tpep_pickup_datetime        0
tpep_dropoff_datetime       0
passenger_count          6077
trip_distance               0
RatecodeID               6077
store_and_fwd_flag       6077
PULocationID                0
DOLocationID                0
payment_type                0
extra                       0
tip_amount                  0
tolls_amount                0
improvement_surcharge       0
total_amount                0
congestion_surcharge     6077
Airport_fee              6077
dtype: int64

In [5]:
train_pd = copy_train_pd.copy()

y = train_pd['total_amount']
X = train_pd.drop('total_amount', axis=1) # seprating prediction element from training set
list_of_features = list(train_pd)


In [6]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
x_train_copy = x_train.copy()
x_test_copy = x_test.copy()


## Preprocessing section
* selecting the features for imputation
* transform each feature into same scale
* plotting each feature with respect to each one
* finding the corelation between features

In [7]:
print('Null values in the given training data set is:')
x_train.isna().sum()

Null values in the given training data set is:


VendorID                    0
tpep_pickup_datetime        0
tpep_dropoff_datetime       0
passenger_count          4826
trip_distance               0
RatecodeID               4826
store_and_fwd_flag       4826
PULocationID                0
DOLocationID                0
payment_type                0
extra                       0
tip_amount                  0
tolls_amount                0
improvement_surcharge       0
congestion_surcharge     4826
Airport_fee              4826
dtype: int64

#### Checking unique values of nan elements

In [8]:
x_train_copy.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,extra,tip_amount,tolls_amount,improvement_surcharge,congestion_surcharge,Airport_fee
26258,1,2023-06-28 21:06:42,2023-06-28 21:05:09,2.0,1.32,1.0,N,127,236,Credit Card,1.0,7.648582,0.0,1.0,2.5,0.00
139643,1,2023-06-30 21:48:21,2023-06-30 22:25:33,1.0,4.01,1.0,N,113,20,Credit Card,1.0,8.690932,0.0,1.0,0.0,1.75
120117,1,2023-06-29 13:21:32,2023-06-29 12:36:33,2.0,1.19,1.0,N,244,113,Cash,0.0,4.989504,0.0,1.0,2.5,0.00
87794,1,2023-06-29 17:28:55,2023-06-29 19:11:52,1.0,1.00,1.0,N,239,197,Credit Card,2.5,5.865453,0.0,1.0,2.5,0.00
113501,1,2023-06-30 10:32:51,2023-06-30 09:18:40,1.0,1.24,1.0,N,95,155,Credit Card,0.0,2.994578,0.0,1.0,2.5,0.00


In [9]:
print(f'''
unique values of passanger_count is: {x_train['passenger_count'].unique()}\n
unique values of RateCodeId is: {x_train['RatecodeID'].unique()}\n
unique values of store_and_fwd_flag is: {x_train['store_and_fwd_flag'].unique()}\n
unique values of congestion_surcharge is: {x_train['congestion_surcharge'].unique()}\n
unqiue values of airport_fees is: {x_train['Airport_fee'].unique()}\n
''')




unique values of passanger_count is: [ 2.  1.  0. nan  4.  5.  3.  6.  9.  8.]

unique values of RateCodeId is: [ 1. nan  3.  2.  4.  5. 99.]

unique values of store_and_fwd_flag is: ['N' nan 'Y']

unique values of congestion_surcharge is: [ 2.5  0.   nan -2.5]

unqiue values of airport_fees is: [ 0.    1.75   nan -1.75]




#### Seprating number values and categorical variables

### Creating pipelines for data processing

##### converting datetime into date time object

In [10]:
def convert_dt_obj_to_datetime(df:pd.DataFrame, col_name:str):
    df[col_name] = pd.to_datetime(df[col_name])
    df[col_name +'_Year'] = df[col_name].apply(lambda time: time.year)
    df[col_name +'_Month'] = df[col_name].apply(lambda time: time.month)
    df[col_name +'_Day'] = df[col_name].apply(lambda time: time.day)
    df[col_name +'_Hour'] = df[col_name].apply(lambda time: time.hour)
#     df[col_name +'_Minute'] = df[col_name].apply(lambda time: time.minute) # IGNORING MINS

In [11]:
x_train, x_test = x_train_copy.copy(), x_test_copy.copy()
  
convert_dt_obj_to_datetime(x_train, 'tpep_pickup_datetime')
convert_dt_obj_to_datetime(x_test, 'tpep_pickup_datetime')
x_train = x_train.drop('tpep_pickup_datetime', axis= 1)
x_test = x_test.drop('tpep_pickup_datetime', axis= 1)

convert_dt_obj_to_datetime(x_train, 'tpep_dropoff_datetime')
convert_dt_obj_to_datetime(x_test, 'tpep_dropoff_datetime')

x_train = x_train.drop('tpep_dropoff_datetime', axis= 1)
x_test = x_test.drop('tpep_dropoff_datetime', axis= 1)

x_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 140000 entries, 26258 to 92409
Data columns (total 22 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   VendorID                     140000 non-null  int64  
 1   passenger_count              135174 non-null  float64
 2   trip_distance                140000 non-null  float64
 3   RatecodeID                   135174 non-null  float64
 4   store_and_fwd_flag           135174 non-null  object 
 5   PULocationID                 140000 non-null  int64  
 6   DOLocationID                 140000 non-null  int64  
 7   payment_type                 140000 non-null  object 
 8   extra                        140000 non-null  float64
 9   tip_amount                   140000 non-null  float64
 10  tolls_amount                 140000 non-null  float64
 11  improvement_surcharge        140000 non-null  float64
 12  congestion_surcharge         135174 non-null  float64
 13  A


Creating pipeline for remaining variable

In [12]:
# tip_amount = Pipeline([
#     ('standard_scaler', StandardScaler())
# ])

congestion_charger_pipe = Pipeline([
    ('simple_imputer', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value= 0)),
#     ('standard_scaler', StandardScaler())
])

airport_fee_pipe = Pipeline([
    ('simple_imputer', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value= 0)),
#     ('standard_scaler', StandardScaler())
])

# extra_pipeline = Pipeline([
#     ('standard_scaler', StandardScaler())
# ])

# pu_location_pipe = Pipeline([
#     ('standard_scaler', StandardScaler())
# ])
# do_location_pipe = Pipeline([
#     ('standard_scaler', StandardScaler())
# ])


rate_code_id_pipe = Pipeline([
     ('simple_imputer', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value= 1)),
#     ('standard_scaler', StandardScaler())
])
store_and_fwd_pipe = Pipeline([
    ('simple_immmputer', SimpleImputer(missing_values= np.nan, strategy='constant', fill_value='N')),
    ('one_hot_encoder', OneHotEncoder())
])
payment_type_pipe = Pipeline([
    ('one_hot_encoder', OneHotEncoder())
])
passanger_count_pipe = Pipeline([
    ('simple_imputer', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value= 0)),
#     ('standard_scaler', StandardScaler())
])



creating column transformers

In [13]:
'''
0    VendorID                     140000 non-null  int64  
 1   passenger_count              135153 non-null  float64
 2   trip_distance                140000 non-null  float64
 3   RatecodeID                   135153 non-null  float64
 4   store_and_fwd_flag           135153 non-null  object 
 5   PULocationID                 140000 non-null  int64  
 6   DOLocationID                 140000 non-null  int64  
 7   payment_type                 140000 non-null  object 
 8   extra                        140000 non-null  float64
 9   tip_amount                   140000 non-null  float64
 10  tolls_amount                 140000 non-null  float64
 11  improvement_surcharge        140000 non-null  float64
 12  congestion_surcharge         135153 non-null  float64
 13  Airport_fee                  135153 non-null  float64
 14  tpep_pickup_datetime_Year    140000 non-null  int64  
 15  tpep_pickup_datetime_Month   140000 non-null  int64  
 16  tpep_pickup_datetime_Day     140000 non-null  int64  
 17  tpep_pickup_datetime_Hour    140000 non-null  int64  
 18  tpep_dropoff_datetime_Year   140000 non-null  int64  
 19  tpep_dropoff_datetime_Month  140000 non-null  int64  
 20  tpep_dropoff_datetime_Day    140000 non-null  int64  
 21  tpep_dropoff_datetime_Hour   140000 non-null  int64
'''


main_pipeline = ColumnTransformer([
#     ('vendor_id',  StandardScaler(), "VendorID"),
    ("passanger_count_t", passanger_count_pipe, ["passenger_count"]),
    ('trip_distance_t', StandardScaler(), ['trip_distance']),
    ('rate_code_id', rate_code_id_pipe, ['RatecodeID'] ),
    ('s_nd_f_flag', store_and_fwd_pipe, ['store_and_fwd_flag']),
    ('pu_loc', StandardScaler(), ['PULocationID']),
    ('du_loc', StandardScaler(), ["DOLocationID"]),
    ('payment_t', payment_type_pipe, ["payment_type"]),
    ('extra_t', StandardScaler(), ['extra']),
    ('tip_amoun_t', StandardScaler(), ['tip_amount']),
    ('tolls_amount_t', StandardScaler(), ["tolls_amount"]),
    ('improvement_c', StandardScaler(), ['improvement_surcharge']),
    ('cong_charge', congestion_charger_pipe, ['congestion_surcharge']),
    ('Airport_fee_t', airport_fee_pipe, ['Airport_fee']),
    ('std_scaler', StandardScaler(), [i for i in range(14, 22)])
], remainder= 'passthrough' )
main_pipeline

ColumnTransformer(remainder='passthrough',
                  transformers=[('passanger_count_t',
                                 Pipeline(steps=[('simple_imputer',
                                                  SimpleImputer(fill_value=0,
                                                                strategy='constant'))]),
                                 ['passenger_count']),
                                ('trip_distance_t', StandardScaler(),
                                 ['trip_distance']),
                                ('rate_code_id',
                                 Pipeline(steps=[('simple_imputer',
                                                  SimpleImputer(fill_value=1,
                                                                strategy='constant'))]),
                                 ['RatecodeI...
                                ('improvement_c', StandardScaler(),
                                 ['improvement_surcharge']),
                                ('cong_charge',
                                 Pipeline(steps=[('simple_imputer',
                                                  SimpleImputer(fill_value=0,
                                                                strategy='constant'))]),
                                 ['congestion_surcharge']),
                                ('Airport_fee_t',
                                 Pipeline(steps=[('simple_imputer',
                                                  SimpleImputer(fill_value=0,
                                                                strategy='constant'))]),
                                 ['Airport_fee']),
                                ('std_scaler', StandardScaler(),
                                 [14, 15, 16, 17, 18, 19, 20, 21])])

In [14]:
main_pipeline.fit(x_train)
x_train_t = main_pipeline.transform(x_train)
x_train_t = pd.DataFrame(x_train_t)
print(x_train_t.isna().sum())

x_train_t.head()
# x_train.info()


0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
dtype: int64


,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,2.0,-0.009494,1.0,1.0,0.0,-0.074681,1.354266,0.0,1.0,0.0,...,0.00,0.0,-0.06637,-0.528181,1.019949,0.0,-0.088459,-0.375674,0.991882,1.0
1,1.0,-0.003402,1.0,1.0,0.0,-0.258407,-1.479948,0.0,1.0,0.0,...,1.75,0.0,-0.06637,0.470541,1.019949,0.0,-0.088459,0.399408,1.159888,1.0
2,2.0,-0.009788,1.0,1.0,0.0,1.460742,-0.259662,1.0,0.0,0.0,...,0.00,0.0,-0.06637,-0.028820,-0.365643,0.0,-0.088459,0.011867,-0.520168,1.0
3,1.0,-0.010218,1.0,1.0,0.0,1.395125,0.842532,0.0,1.0,0.0,...,0.00,0.0,-0.06637,-0.028820,0.327153,0.0,-0.088459,0.011867,0.655871,1.0
4,1.0,-0.009675,1.0,1.0,0.0,-0.494626,0.291435,0.0,1.0,0.0,...,0.00,0.0,-0.06637,0.470541,-0.885240,0.0,-0.088459,0.399408,-1.024185,1.0


# checking data using leanear regression

In [15]:
lr = LinearRegression()
lr.fit(X=x_train_t, y= y_train)
print('mse after basic transformation is:', mean_squared_error(y_train, lr.predict(x_train_t)))
print('train score is:', lr.score(x_train_t, y_train))

mse after basic transformation is: 176.86258918781778
train score is: 0.7277026084951077


test score

In [16]:
x_test_t = main_pipeline.fit_transform(x_test)
x_test_t = pd.DataFrame(x_test_t)
x_test_t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35000 entries, 0 to 34999
Data columns (total 27 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       35000 non-null  float64
 1   1       35000 non-null  float64
 2   2       35000 non-null  float64
 3   3       35000 non-null  float64
 4   4       35000 non-null  float64
 5   5       35000 non-null  float64
 6   6       35000 non-null  float64
 7   7       35000 non-null  float64
 8   8       35000 non-null  float64
 9   9       35000 non-null  float64
 10  10      35000 non-null  float64
 11  11      35000 non-null  float64
 12  12      35000 non-null  float64
 13  13      35000 non-null  float64
 14  14      35000 non-null  float64
 15  15      35000 non-null  float64
 16  16      35000 non-null  float64
 17  17      35000 non-null  float64
 18  18      35000 non-null  float64
 19  19      35000 non-null  float64
 20  20      35000 non-null  float64
 21  21      35000 non-null  float64
 22

In [17]:
print('test mean squared error is:', mean_squared_error(y_test, lr.predict(x_test_t)))
print('test score is:', lr.score(x_test_t, y_test))

test mean squared error is: 174.53671017824703
test score is: 0.7247492070777921


### Submission code

In [18]:
submission = pd.DataFrame(columns= ['ID', "total_amount"])
submission['ID'] = [i for i in range(1, 50001)]
submission['total_amount'] = [i for i in range(50000)]
submission.to_csv('submission.csv', index=False)